In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
import scipy
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('../Data/cat.csv')
df = df.drop('id', axis=1)
df.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [3]:
dtarget = df['target']
df = df.drop('target', axis=1)
df = df.drop('nom_5', axis=1)
df = df.drop('nom_6', axis=1)
df = df.drop('nom_7', axis=1)
df = df.drop('nom_8', axis=1)
df = df.drop('nom_9', axis=1)

In [4]:
dt = pd.get_dummies(df, columns=df.columns, sparse=True)
#dt = dt.sparse.to_coo()
#dt = dt.tocsr()
dt.shape

(300000, 301)

In [5]:
size = 0.8
np.random.seed(123)
ids = np.random.rand(dt.shape[0]) < size #list(np.where(np.random.rand(dt.shape[0]) < size)[0])

In [6]:
x = dt[ids]; y = dtarget[ids]
x_val = dt[~ids]; y_val = dtarget[~ids]

In [7]:
model = LogisticRegression(C=0.1, class_weight='balanced',max_iter=10000, solver='lbfgs', n_jobs=-1)
model.fit(x, y)

LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
from sklearn import metrics

pred = model.predict_proba(x)
print(metrics.roc_auc_score(y, pred[:,1]))

pred = model.predict_proba(x_val)
metrics.roc_auc_score(y_val, pred[:,1])

0.7679087956854856


0.769353332206104

## pytorch LogisticRegression

In [9]:
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
Tx = torch.from_numpy(x.values.astype(np.float32))
Ty = torch.from_numpy(y.values.astype(np.int64)).flatten() 

Tx_val = torch.from_numpy(x_val.values.astype(np.float32))
Ty_val = torch.from_numpy(y_val.values.astype(np.int64)).flatten()

In [11]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [12]:
criterion = torch.nn.CrossEntropyLoss() # computes softmax and then the cross entropy

model = LogisticRegression(x.shape[1], 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.1)

In [13]:
epochs = 1000
for i in range(epochs):
    
    pred = model(Tx)
    loss = criterion(pred, Ty)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100==0:
        with torch.set_grad_enabled(False):
            auc_train = metrics.roc_auc_score(y, torch.nn.functional.softmax(pred, dim=1).data.numpy()[:,1])

            pred_val = model(Tx_val)
            auc_val = metrics.roc_auc_score(y_val, torch.nn.functional.softmax(pred_val, dim=1).data.numpy()[:,1])
            print(loss.item(), auc_train, auc_val)

0.7246635556221008 0.46707843656980974 0.48500814742552095
0.5731556415557861 0.7167675010797456 0.7187511243813368
0.5679134130477905 0.7231539553712348 0.7250674203901032
0.5669075846672058 0.7248349870559174 0.7267522449241384
0.5666638016700745 0.7253293112856511 0.7272479589024776
0.5666008591651917 0.7254770830834246 0.7273939209435683
0.5665844082832336 0.7255220845564845 0.7274383991677058
0.5665791034698486 0.7255359986055389 0.7274524372957364
0.566578209400177 0.7255403301947796 0.7274565335256307
0.5665776133537292 0.7255417599473396 0.7274579262569087


## Logistic2

In [24]:
Tx = torch.from_numpy(x.values.astype(np.float32))
Ty = torch.from_numpy(y.values.astype(np.float32)).flatten() 

Tx_val = torch.from_numpy(x_val.values.astype(np.float32))
Ty_val = torch.from_numpy(y_val.values.astype(np.float32)).flatten()

In [49]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.linear(x)
        outputs = torch.sigmoid(x)
        return outputs

In [61]:
criterion = torch.nn.BCELoss()

model = LogisticRegression(x.shape[1], 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.1)

In [62]:
epochs = 1000
for i in range(epochs):
    
    pred = model(Tx)
    loss = criterion(pred.view(-1), Ty)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100==0:
        with torch.set_grad_enabled(False):
            auc_train = metrics.roc_auc_score(y, pred.data.numpy())

            pred_val = model(Tx_val)
            auc_val = metrics.roc_auc_score(y_val, pred_val.data.numpy())
            print(loss.item(), auc_train, auc_val)

0.6957900524139404 0.5504324574378088 0.5634266066103454
0.5861526131629944 0.7113497532824442 0.7133566079625275
0.581739068031311 0.7157441797212254 0.7174870096071863
0.5807616710662842 0.7168324526084804 0.7185280001751642
0.5805249810218811 0.7171971803576777 0.7188788189550632
0.5804644227027893 0.7173195633931162 0.7189972214407744
0.5804470777511597 0.717360127529754 0.7190355025352928
0.580442488193512 0.7173733566329034 0.719048361694002
0.5804405808448792 0.717377684258668 0.719052397598436
0.580440104007721 0.7173791180564253 0.7190538447537709


## complex model

In [79]:
class TabularModel(nn.Module):

    def __init__(self, in_sz, out_sz, layers, p=0.5):
        super().__init__()
        
        layerlist = []
        for i in layers:
            layerlist.append(nn.Linear(in_sz,i)) 
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            in_sz = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
            
        self.layers = nn.Sequential(*layerlist)
    
    def forward(self, x):
        
        x = self.layers(x)
        return x

In [80]:
criterion = torch.nn.CrossEntropyLoss() # computes softmax and then the cross entropy

model = TabularModel(x.shape[1], 2, [10, 10])
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.1)

In [81]:
epochs = 1000
for i in range(epochs):
    
    pred = model(Tx)
    loss = criterion(pred, Ty)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100==0:
        with torch.set_grad_enabled(False):
            auc_train = metrics.roc_auc_score(y, torch.nn.functional.softmax(pred, dim=1).data.numpy()[:,1])

            pred_val = model(Tx_val)
            auc_val = metrics.roc_auc_score(y_val, torch.nn.functional.softmax(pred_val, dim=1).data.numpy()[:,1])
            print(loss.item(), auc_train, auc_val)

0.8469741940498352 0.5202347236549539 0.5358845659413436
0.5672557950019836 0.7135237744451493 0.7162017002694255
0.5599502325057983 0.7348054158716102 0.7363792389120662
0.5617085695266724 0.7347693200437144 0.7371969354445866
0.5599080324172974 0.7375465074796345 0.7394102641231726
0.5587478280067444 0.7384434317819042 0.7392128870857253
0.5585008859634399 0.7391016786483443 0.7406561606083233
0.5587579607963562 0.7390643361641482 0.738626860647678
0.5589296221733093 0.7382515139273162 0.740064689797487
0.5585237145423889 0.7393658129610957 0.741134949360789
